In [ ]:
import sys
from pathlib import Path

AVES_ROOT = Path("../..") if not "google.colab" in sys.modules else Path("aves_git")

EOD_PATH = AVES_ROOT / "data" / "external" / "EOD_STGO"
EOD_PATH

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import geopandas as gpd

# esto configura la calidad de la imagen. dependerá de tu resolución. el valor por omisión es 80
mpl.rcParams["figure.dpi"] = 120
# esto depende de las fuentes que tengas instaladas en el sistema.
#mpl.rcParams["font.family"] = "Fira Sans Extra Condensed"


In [ ]:
zones = gpd.read_file(AVES_ROOT / "data" / "processed" / "scl_zonas_urbanas.json")
zones.head()

In [ ]:
from aves.data import census

comunas = census.read_census_map('comuna', path=AVES_ROOT / "data" / "external" / "censo_2017_R13")

In [ ]:
comunas_urbanas = comunas[comunas['COMUNA'].isin(zones['Com'].unique())].drop('NOM_COMUNA', axis=1).copy()
comunas_urbanas['NombreComuna'] = comunas_urbanas['COMUNA'].map(dict(zip(zones['Com'], zones['Comuna'])))
comunas_urbanas.plot(facecolor="none", edgecolor="#abacab")

In [ ]:
from aves.features.geo import clip_area_geodataframe
comunas_urbanas = clip_area_geodataframe(comunas_urbanas, zones.total_bounds, buffer=0.05)
comunas_urbanas.plot()

In [ ]:
from aves.models.network import Network


In [ ]:
from aves.visualization.networks import NodeLink


In [ ]:
from aves.visualization.figures import figure_from_geodataframe



In [ ]:
import pyrosm

# redes con OSM
OSM_PATH = AVES_ROOT / "data" / "external" / "OSM"
osm = pyrosm.OSM(str(OSM_PATH / 'clipped-scl-osm.pbf'))

In [ ]:
network_nodes, network_edges  = osm.get_network(nodes=True, network_type='driving')

In [ ]:
network_nodes

In [ ]:
network_edges

In [ ]:
network_edges[['u', 'v', 'length']]

In [ ]:
walk_network = Network.from_edgelist(
    network_edges,
    source="u",
    target="v",
    weight="length",
)

In [ ]:
walk_nodelink = NodeLink(walk_network)
walk_nodelink.layout_nodes(
    method="geographical", geodataframe=network_nodes, node_column="id"
)


In [ ]:
fig, ax = figure_from_geodataframe(zones, height=7)

comunas_urbanas.plot(ax=ax, facecolor="none", edgecolor="#abacab", zorder=1)

walk_nodelink.plot(
    ax,
    nodes=dict(color="black", edgecolor="none", node_size=1, alpha=1.0),
    edges=dict(alpha=0.25, linewidth=0.25),
    zorder=2,
)

fig.tight_layout()


In [ ]:
scl_nodes = gpd.sjoin(
    network_nodes, zones[zones["NOM_COMUNA"] == "SANTIAGO"], op="within"
)
scl_nodes

In [ ]:
scl_network = walk_network.subgraph(nodes=scl_nodes['id'].values)

In [ ]:
scl_network.graph

In [ ]:
scl_nodelink = NodeLink(scl_network)
scl_nodelink.layout_nodes(
    method="geographical", geodataframe=network_nodes, node_column="id"
)

In [ ]:
fig, ax = figure_from_geodataframe(zones[zones["NOM_COMUNA"] == "SANTIAGO"], height=7)

comunas_urbanas.plot(ax=ax, facecolor="none", edgecolor="#abacab", zorder=1)

scl_nodelink.plot(
    ax,
    nodes=dict(color="black", edgecolor="none", node_size=1, alpha=1.0),
    edges=dict(alpha=0.5, linewidth=0.25),
    zorder=2,
)

fig.tight_layout()


In [ ]:
pagerank = scl_network.estimate_pagerank()
pagerank

In [ ]:
np.array(pagerank.a).shape

In [ ]:
pd.Series(pagerank.a).plot(kind='hist', bins=100)

In [ ]:
scl_nodelink.set_node_drawing('plain', weights=np.power(np.array(pagerank.a), 2))

In [ ]:
fig, ax = figure_from_geodataframe(zones[zones["NOM_COMUNA"] == "SANTIAGO"], height=7)

comunas_urbanas.plot(ax=ax, facecolor="none", edgecolor="#abacab", zorder=1)

scl_nodelink.plot(
    ax,
    nodes=dict(color="black", edgecolor="none", node_size=100, alpha=0.75),
    edges=dict(alpha=0.5, linewidth=0.25),
    zorder=2,
)

fig.tight_layout()

In [ ]:
# centralidad


In [ ]:
node_betweenness, edge_betweenness = scl_network.estimate_betweenness()

In [ ]:
scl_nodelink.set_node_drawing('plain', weights=np.power(np.array(node_betweenness.a), 2))

In [ ]:
scl_nodelink.set_edge_drawing('weighted', weights=np.power(np.array(edge_betweenness.a), 2))

In [ ]:
fig, ax = figure_from_geodataframe(zones[zones["NOM_COMUNA"] == "SANTIAGO"], height=7)

comunas_urbanas.plot(ax=ax, facecolor="none", edgecolor="#abacab", zorder=1)

scl_nodelink.plot(
    ax,
    nodes=dict(color="black", edgecolor="none", node_size=10, alpha=0.75),
    edges=dict(alpha=0.95, linewidth=2, palette='Greys', ),
    zorder=2,
)

fig.tight_layout()

In [ ]:
# camino más corto
src_point = (-70.6651616, -33.4582698)
dst_point = (-70.6397468, -33.4391656)


In [ ]:
import shapely.geometry

def get_node_id(query_point):
    query_point_node_id = scl_nodes.distance(
        shapely.geometry.Point(query_point)
    ).idxmin()
    return scl_nodes.loc[query_point_node_id]['id']


In [ ]:
src_node_id = get_node_id(src_point)
dst_node_id = get_node_id(dst_point)

src_node_id, dst_node_id

In [ ]:
path = scl_network.shortest_path(src_node_id, dst_node_id)
len(path)

In [ ]:
len(path[0])

In [ ]:
path_network = scl_network.subgraph(nodes=path[0])
path_nodelink = NodeLink(path_network)
path_nodelink.layout_nodes(
    method="geographical", geodataframe=network_nodes, node_column="id"
)

In [ ]:
fig, ax = figure_from_geodataframe(zones[zones["NOM_COMUNA"] == "SANTIAGO"], height=7)

comunas_urbanas.plot(ax=ax, facecolor="none", edgecolor="#abacab", zorder=1)

scl_nodelink.plot(
    ax,
    nodes=dict(color="black", edgecolor="none", node_size=10, alpha=0.75),
    edges=dict(alpha=0.95, linewidth=2, palette='Greys', ),
    zorder=2,
)

path_nodelink.plot(
    ax,
    nodes=dict(color="black", edgecolor="none", node_size=1, alpha=0.75),
    edges=dict(alpha=0.95, linewidth=5, color='red'),
    zorder=2,
)

fig.tight_layout()


In [ ]:
path = scl_network.shortest_path(src_node_id, dst_node_id, weights=scl_network._edge_weight)
len(path)

In [ ]:
path_network = scl_network.subgraph(nodes=path[0])
path_nodelink = NodeLink(path_network)
path_nodelink.layout_nodes(
    method="geographical", geodataframe=network_nodes, node_column="id"
)

In [ ]:
fig, ax = figure_from_geodataframe(zones[zones["NOM_COMUNA"] == "SANTIAGO"], height=7)

comunas_urbanas.plot(ax=ax, facecolor="none", edgecolor="#abacab", zorder=1)

scl_nodelink.plot(
    ax,
    nodes=dict(color="black", edgecolor="none", node_size=10, alpha=0.75),
    edges=dict(alpha=0.95, linewidth=2, palette='Greys', ),
    zorder=2,
)

path_nodelink.plot(
    ax,
    nodes=dict(color="black", edgecolor="none", node_size=1, alpha=0.75),
    edges=dict(alpha=0.95, linewidth=5, color='red'),
    zorder=2,
)

fig.tight_layout()

In [ ]:
import graph_tool.topology

min_tree = graph_tool.topology.min_spanning_tree(scl_network.graph, weights=scl_network._edge_weight)
min_tree

In [ ]:
min_tree_network = scl_network.subgraph(edge_filter=min_tree)
min_tree_network.graph

In [ ]:
tree_nodelink = NodeLink(min_tree_network)
tree_nodelink.layout_nodes(
    method="geographical", geodataframe=network_nodes, node_column="id"
)

In [ ]:
fig, ax = figure_from_geodataframe(zones[zones["NOM_COMUNA"] == "SANTIAGO"], height=7)

comunas_urbanas.plot(ax=ax, facecolor="none", edgecolor="#abacab", zorder=1)

scl_nodelink.plot(
    ax,
    nodes=dict(color="black", edgecolor="none", node_size=10, alpha=0.75),
    edges=dict(alpha=0.95, linewidth=2, palette='Greys', ),
    zorder=2,
)

tree_nodelink.plot(
    ax,
    nodes=dict(color="black", edgecolor="none", node_size=1, alpha=0.75),
    edges=dict(alpha=0.95, linewidth=5, color='red'),
    zorder=2,
)

fig.tight_layout()